In [ ]:
#Code for plot_components() in Plotly

In [ ]:
def plot_components(m, forecast, uncertainty=True, plot_cap=True, weekly_start=0, yearly_start=0):
    '''  
    Will plot whichever are available of: trend, holidays, weekly seasonality, yearly seasonality, 
    and additive and multiplicative extra regressors. 
    Returns a df with all axes values for the corresponding plots
    Refer source code - https://github.com/facebook/prophet/blob/master/python/fbprophet/plot.py
    '''

    components = ['trend']

    if m.train_holiday_names is not None and 'holidays' in forecast:
            components.append('holidays')
        # Plot weekly seasonality, if present
    if 'weekly' in m.seasonalities and 'weekly' in forecast:
        components.append('weekly')
    # Yearly if present
    if 'yearly' in m.seasonalities and 'yearly' in forecast:
        components.append('yearly')
    # Other seasonalities
    components.extend([
        name for name in sorted(m.seasonalities)
        if name in forecast and name not in ['weekly', 'yearly']
    ])
    regressors = {'additive': False, 'multiplicative': False}
    for name, props in m.extra_regressors.items():
        regressors[props['mode']] = True
    for mode in ['additive', 'multiplicative']:
        if regressors[mode] and 'extra_regressors_{}'.format(mode) in forecast:
            components.append('extra_regressors_{}'.format(mode))


    for plot_name in components:
        if plot_name == 'trend':
            pass
            plot_forecast_component(
                m=m, forecast=forecast, name='trend', uncertainty=uncertainty, plot_cap=plot_cap,
            )
        elif plot_name == 'weekly':
            plot_weekly(
                m=m, uncertainty=uncertainty, weekly_start=weekly_start,
            )
        elif plot_name == 'yearly':
            plot_yearly(
                m=m, uncertainty=uncertainty, yearly_start=yearly_start,
            )
        elif plot_name in [
            'holidays',
            'extra_regressors_additive',
            'extra_regressors_multiplicative',
        ]:
            plot_forecast_component(
                m=m, forecast=forecast, name=plot_name, uncertainty=uncertainty,plot_cap=False,
            )
        else:
            plot_seasonality(
                m=m, name=plot_name,  uncertainty=uncertainty,
            )


def plot_forecast_component(m, forecast, name, uncertainty=True, plot_cap=False):
    '''
    Get axes values to plot a particular component of the forecast
    '''
    fcst_t = forecast['ds'].dt.to_pydatetime()
    data = []

    trace1 = go.Scatter( x=fcst_t, y=forecast[name], line = dict(color = '#2164F3'), name=name,  showlegend = False,)
    data.append(trace1)

    if 'cap' in forecast and plot_cap:
        trace2 = go.Scatter( x=fcst_t, y=forecast['cap'], line = dict(dash = 'dash', color = 'rgb(0, 0, 0)'))
        data.append(trace2)
    if m.logistic_floor and 'floor' in forecast and plot_cap:
        trace3 = go.Scatter( x=fcst_t, y=forecast['floor'], line = dict(dash = 'dash', color = 'rgb(0, 0, 0)'))
        data.append(trace3)
    if uncertainty:
        date_list = forecast['ds'].dt.to_pydatetime().astype('str').tolist()
        date_list_reverse = date_list[::-1]

        trace4 = go.Scatter(
            x=date_list + date_list_reverse,
            y=forecast[name + '_upper'].tolist() + forecast[name + '_lower'].tolist() [::-1],
            fill='tozerox',
            fillcolor='#4CBBF3',
            name='u-l bound',
            showlegend = False,
            line = dict(color = '#2164F3')
        )

        data.append(trace4)

    if name in m.component_modes['multiplicative']:
        layout = go.Layout(yaxis=dict(title = name, tickformat='%'), xaxis=dict(title='ds'),width=700,height=500,)
    else:
        layout = go.Layout(xaxis=dict(title='ds'), yaxis=dict(title=name),width=700,height=500,)

    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig,  filename='try')
    

def seasonality_plot_df(m, ds):
    '''
    Prepare dataframe for plotting seasonal components
    '''
    df_dict = {'ds': ds, 'cap': 1., 'floor': 0.}
    for name in m.extra_regressors:
        df_dict[name] = 0.
    df = pd.DataFrame(df_dict)
    df = m.setup_dataframe(df)
    return df


def plot_weekly(m, uncertainty=True, weekly_start=0):
    '''
    Get axes values to plot the weekly component of the forecast.
    '''
    uncertainty=True
    weekly_start=0

    data = []

    days = (pd.date_range(start='2017-01-01', periods=7) + pd.Timedelta(days=weekly_start))
    df_w = seasonality_plot_df(m, days)
    seas = m.predict_seasonal_components(df_w)
    days = days.weekday_name
    
    


    trace1 = go.Scatter(x=list(range(len(days))), y=seas['weekly'].tolist(), line = dict(color = '#2164F3'), showlegend=False, name='weekly')
    data.append(trace1)

    if uncertainty:
        x_list = list(range(len(days)))
        x_list_reverse = x_list[::-1]

        trace2 = go.Scatter(
            x= x_list + x_list_reverse,
            y=seas['weekly_upper'].tolist() + seas['weekly_lower'].tolist() [::-1],
            fill='tozerox',
            fillcolor='#4CBBF3',
            showlegend=False,
            name='u-l bound',
            line = dict(color = '#2164F3')
        )

        data.append(trace2)



    if m.seasonalities['weekly']['mode'] == 'multiplicative':
        layout = go.Layout(width=700, height=500,
            yaxis=dict(title = 'weekly', tickformat='%'), 
            xaxis=dict(title = 'Day of week',
                                  ticktext=days, 
                                  tickvals=list(range(len(days)))))
    else:
        layout = go.Layout(xaxis=dict(title = 'Day of week',ticktext=days, tickvals=list(range(len(days)))), yaxis=dict(title='weekly'), width=700,height=500,)

    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig,  filename='try')
    
    

def plot_yearly(m, uncertainty=True, yearly_start=0):
    '''
    Get axes values to plot the yearly component of the forecast.
    '''
    uncertainty=True
    yearly_start=0

    data = []


    days = (pd.date_range(start='2017-01-01', periods=365) + pd.Timedelta(days=yearly_start))
    df_y = seasonality_plot_df(m, days)
    seas = m.predict_seasonal_components(df_y)

    trace1 = go.Scatter(x=df_y['ds'].dt.to_pydatetime(), y=seas['yearly'], line = dict(color = '#2164F3'), name='yearly', showlegend=False)
    data.append(trace1)

    if uncertainty:
        date_list = df_y['ds'].dt.to_pydatetime().astype('str').tolist()
        date_list_reverse = date_list[::-1]

        trace2 = go.Scatter(
            x= date_list + date_list_reverse,
            y=seas['yearly_upper'].tolist() + seas['yearly_lower'].tolist() [::-1],
            fill='tozerox',
            fillcolor='#4CBBF3',
            showlegend=False,
            name='u-l bound',
            line = dict(color = '#2164F3')
        )

        data.append(trace2)


    if m.seasonalities['yearly']['mode'] == 'multiplicative':
        layout = go.Layout(yaxis=dict(title = 'yearly', tickformat='%'), xaxis=dict(title = 'Day of year'),width=700,
    height=500,)
    else:
        layout = go.Layout(yaxis=dict(title='yearly'),  xaxis=dict(title = 'Day of year'),width=700,
    height=500,)



    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig,  filename='try')

    
def plot_seasonality(m, name, uncertainty=True):
    '''
    Get axes values to plot a custom seasonal component 
    '''

    data = []

    start = pd.to_datetime('2017-01-01 0000')
    period = m.seasonalities[name]['period']
    end = start + pd.Timedelta(days=period)
    plot_points = 200
    days = pd.to_datetime(np.linspace(start.value, end.value, plot_points))
    df_y = seasonality_plot_df(m, days)
    seas = m.predict_seasonal_components(df_y)



    trace1 = go.Scatter(x=df_y['ds'].dt.to_pydatetime(), y=seas[name], line = dict(color = '#2164F3'), showlegend=False, name=name)
    data.append(trace1)


    if uncertainty:
        date_list = df_y['ds'].dt.to_pydatetime().astype('str').tolist()
        date_list_reverse = date_list[::-1]

        trace2 = go.Scatter(
            x= date_list + date_list_reverse,
            y=seas[name + '_upper'].tolist() + seas[name + '_lower'].tolist() [::-1],
            fill='tozerox',
            fillcolor='#4CBBF3',
            showlegend=False,
            name='u-l bound',
            line = dict(color = '#2164F3')
        )

        data.append(trace2)

    if m.seasonalities[name]['mode'] == 'multiplicative':
        layout = go.Layout(yaxis=dict(title = name, tickformat='%'), xaxis=dict(title = 'ds'))
    else:
        layout = go.Layout(yaxis=dict(title=name),  xaxis=dict(title = 'ds'))

    plotly.offline.init_notebook_mode(connected=True)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig,  filename='try')






In [ ]:
# USAGE
plot_components(prophet_model, forecast)